In [1]:
import findspark
findspark.init('C:\Spark\spark-3.0.1-bin-hadoop2.7')

from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('lrex').getOrCreate()

In [2]:
data = spark.read.csv('Ecommerce_Customers.csv',
                      inferSchema=True,
                      header=True)

In [3]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [4]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [7]:
assembler = VectorAssembler(
    inputCols=['Avg Session Length',
               'Time on App',
               'Time on Website',
               'Length of Membership',
               'Yearly Amount Spent'],
    outputCol='features'
)

In [8]:
output = assembler.transform(data)

In [9]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [11]:
final_data = output.select('features', 'Yearly Amount Spent')

In [12]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [13]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                368|
|   mean|  496.1971643980037|
| stddev|  81.07635023103495|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [14]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                132|
|   mean| 508.00350477901554|
| stddev|  73.78305314568338|
|    min| 302.18954780965197|
|    max|  689.7876041747194|
+-------+-------------------+



In [15]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [16]:
lr_model = lr.fit(train_data)

In [17]:
test_results = lr_model.evaluate(test_data)

In [18]:
test_results.residuals.show(5)

+--------------------+
|           residuals|
+--------------------+
|8.981260180007666...|
|3.524291969370097...|
|-4.43378667114302...|
|7.389644451905042...|
|5.115907697472721...|
+--------------------+
only showing top 5 rows



In [19]:
test_results.rootMeanSquaredError

4.1590474037342344e-12

In [20]:
test_results.r2

1.0

In [21]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
unlabeled_data = test_data.select('features')

In [23]:
predictions = lr_model.transform(unlabeled_data)

In [24]:
predictions.show(5)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[31.1239743499119...| 486.9470538397568|
|[31.1280900496166...| 557.2526867470511|
|[31.1695067987115...|427.35653080229724|
|[31.5702008293202...| 545.9454921413975|
|[31.7216523605090...|347.77692663187213|
+--------------------+------------------+
only showing top 5 rows

